#  Protein-Protein Interaction Network

In [3]:
library(igraph)
biogrid = read.delim("./BIOGRID.txt",stringsAsFactors = F)

In [4]:
#View biogrid
names(biogrid)
attach(biogrid)
HSnet = graph.data.frame(data.frame(Entrez.Gene.Interactor.A,Entrez.Gene.Interactor.B),directed=F)

[1] "X.BioGRID.Interaction.ID"     "Entrez.Gene.Interactor.A"    
 [3] "Entrez.Gene.Interactor.B"     "BioGRID.ID.Interactor.A"     
 [5] "BioGRID.ID.Interactor.B"      "Systematic.Name.Interactor.A"
 [7] "Systematic.Name.Interactor.B" "Official.Symbol.Interactor.A"
 [9] "Official.Symbol.Interactor.B" "Synonyms.Interactor.A"       
[11] "Synonyms.Interactor.B"        "Experimental.System"         
[13] "Experimental.System.Type"     "Author"                      
[15] "Pubmed.ID"                    "Organism.Interactor.A"       
[17] "Organism.Interactor.B"        "Throughput"                  
[19] "Score"                        "Modification"                
[21] "Phenotypes"                   "Qualifications"              
[23] "Tags"                         "Source.Database"

In [5]:
# You can uncomment the following line to plot the network, but it takes a long time
#plot(HSnet)
A = get.adjacency(HSnet)

In [6]:
# multiple edges
A[1:15,1:15]

   [[ suppressing 15 column names '6416', '84665', '90' ... ]]


15 x 15 sparse Matrix of class "dgCMatrix"
                                    
6416  1 . . . . . . . 1 . .  . . 2 .
84665 . . . . . . . . . . .  . . . .
90    . . 1 . . . . . . . .  . . . .
2624  . . . . . . . . . . .  1 . . .
6118  . . . . . . . 1 1 . .  . . . .
375   . . . . . 1 1 . . . .  . . . .
377   . . . . . 1 . . . . .  . . . .
54464 . . . . 1 . . . . . .  . . . .
351   1 . . . 1 . . . 1 . .  . . . .
333   . . . . . . . . . . .  . . . .
10370 . . . . . . . . . . .  8 . . .
2033  . . . 1 . . . . . . 8 17 . . .
338   . . . . . . . . . . .  . 1 . .
409   2 . . . . . . . . . .  . . 1 .
1436  . . . . . . . . . . .  . . . .

In [7]:
# the following is FALSE if the graph is not simple
is.simple(HSnet)

[1] FALSE

In [8]:
# remove multiple edges and self-loops
HSnet = simplify(HSnet, remove.multiple = TRUE, remove.loops = TRUE,
edge.attr.comb = getIgraphOpt("edge.attr.comb"))
is.simple(HSnet)
A = get.adjacency(HSnet)

[1] TRUE

In [9]:
# only single edges now
A[1:15,1:15]

   [[ suppressing 15 column names '6416', '84665', '90' ... ]]


15 x 15 sparse Matrix of class "dgCMatrix"
                                   
6416  . . . . . . . . 1 . . . . 1 .
84665 . . . . . . . . . . . . . . .
90    . . . . . . . . . . . . . . .
2624  . . . . . . . . . . . 1 . . .
6118  . . . . . . . 1 1 . . . . . .
375   . . . . . . 1 . . . . . . . .
377   . . . . . 1 . . . . . . . . .
54464 . . . . 1 . . . . . . . . . .
351   1 . . . 1 . . . . . . . . . .
333   . . . . . . . . . . . . . . .
10370 . . . . . . . . . . . 1 . . .
2033  . . . 1 . . . . . . 1 . . . .
338   . . . . . . . . . . . . . . .
409   1 . . . . . . . . . . . . . .
1436  . . . . . . . . . . . . . . .

In [10]:
# for this application, we remove nodes of very high degree; these are usually house-keeping
# genes that are necessary to keep a cell alive, but are usually not specific to a particular
# disease.
overly.attached.proteins = which(degree(HSnet)>1000)
HSnet = delete.vertices(HSnet, overly.attached.proteins )

In [11]:
# the following is TRUE if the graph is connected.
is.connected(HSnet)

[1] FALSE

#                              Genes Causing Autism

In [12]:
# read the gene-id table
gene.table = read.delim("gene-id-table.txt")
names(gene.table)

[1] "Symbol" "ID"

In [13]:
# read the scores for Autism
gene.score = read.csv("gene-score.csv",stringsAsFactors=F)
attach(gene.score)
names(gene.score)

The following object is masked from biogrid:

    Score



[1] "Gene.Symbol"   "Gene.Name"     "Score.Details" "Score"        
[5] "Evidence"      "References"    "Scored.By"

In [14]:
# display the scores
unique(Score)

[1] "4"  "3"  "1S" "S"  "6"  "5"  "1"  "2S" "2"  "3S" "4S"

In [15]:
# identify the genes that have significant scores
signif.scores = c("3","1S","1","2S","2","3S")
signif.genes = Gene.Symbol[which(Score %in% signif.scores)]
signif.EIDs = gene.table[which(gene.table[,1] %in% signif.genes),2]

In [16]:
# Now use the protein interaction network HSnet, created previously, to determine the genes
# that are present in the network and known to cause Autism
geneEIDs = as.numeric(V(HSnet)$name)
HSnetN = HSnet
V(HSnetN)$name = 1:length(V(HSnet))
signif.ids = which(geneEIDs %in% signif.EIDs)
length(signif.ids)

[1] 100

# Building an Autism Interactome

In [18]:
source('steiner_tree.R')
# Identify the Steiner Tree and note the time this function call takes
system.time(HS.stree <- steiner_tree(terminals=signif.ids, graph=HSnetN))

Warning message in get.shortest.paths(graph, nodes[inds[1, 1]], free_terminals[inds[1, :
"At structural_properties.c:740 :Couldn't reach some vertices"Warning message in get.shortest.paths(graph, nodes[inds[1, 1]], free_terminals[inds[1, :
"At structural_properties.c:740 :Couldn't reach some vertices"Warning message in get.shortest.paths(graph, nodes[inds[1, 1]], free_terminals[inds[1, :
"At structural_properties.c:740 :Couldn't reach some vertices"Warning message in get.shortest.paths(graph, nodes[inds[1, 1]], free_terminals[inds[1, :
"At structural_properties.c:740 :Couldn't reach some vertices"Warning message in get.shortest.paths(graph, nodes[inds[1, 1]], free_terminals[inds[1, :
"At structural_properties.c:740 :Couldn't reach some vertices"

   user  system elapsed 
  36.72    0.25   38.16 

In [19]:
# Output the overlap between significant Autism and vertices in the Steiner Tree
length(intersect(signif.ids,V(HS.stree)$name))
labels = gene.table[as.numeric(V(HS.stree)$name),1]
labels = as.character(labels)

[1] 95

In [20]:
# identify the genes that have significant scores, and assign the color “red” to them
colors = rep("skyblue",length(V(HS.stree)))
colors[which(as.numeric(V(HS.stree)$name) %in% signif.ids)] = "red"

In [21]:
# assign colors to the vertices of the tree
V(HS.stree)$color = colors

In [22]:
# plot and save to file
pdf("ASD_interactome.pdf",width = 12, height = 12)
system.time(plot(HS.stree,vertex.label=labels,vertex.size=5,vertex.label.cex=0.8))
dev.off()

   user  system elapsed 
   0.36    0.00    1.23 

png 
  2

# Analysis and Properties of the Autism Interactome

In [24]:
library(sna, quietly=TRUE)
# a function that computes the connectivity scores for a network
# here the scores are diameters of the network and average geodesic distance between any
# two nodes
c.scores = function(graph) {
n = length(V(graph))
sp = shortest.paths(graph)
neighbors = sum(sp==1)/2
neighbors2 = sum(sp==2)/2
return(c(2*neighbors/(n*(n-1)),2*neighbors2/(n*(n-1))))
}
clus = clusters(HSnetN, mode=c("weak"))
connected.ids = which(clus$membership==1)
length(connected.ids)

[1] 17796

In [25]:
# Generate N randomly chosen subnetworks. Note: this will take a while if N is set large.
N = 50
strees = list(N)
effs = numeric(N)
nei = numeric(N)
nei2 = numeric(N)
for (i in 1:N){
    new.ids<-sample(x=connected.ids,size=length(signif.ids))
    strees[[i]] <- steiner_tree(terminals=new.ids, graph=HSnetN)
    effs[i]<-efficiency(get.adjacency(strees[[i]],sparse=F))
    cs<-c.scores(strees[[i]])
    nei[i]<-cs[1]
    nei2[i]<-cs[2]
}

In [28]:
# print the efficiencies and connectivity scores for each of the N random graphs
effs
nei
nei2

[1] 0.9823663 0.9801899 0.9808830 0.9799254 0.9802419 0.9778161 0.9831550
 [8] 0.9817051 0.9799320 0.9849243 0.9799838 0.9813551 0.9836663 0.9775993
[15] 0.9825379 0.9755464 0.9773373 0.9845679 0.9742745 0.9822509 0.9768519
[22] 0.9815696 0.9784568 0.9795182 0.9751636 0.9833080 0.9846637 0.9850519
[29] 0.9841975 0.9852291 0.9778935 0.9781551 0.9839506 0.9737085 0.9797860
[36] 0.9793426 0.9763042 0.9738272 0.9798765 0.9789652 0.9829878 0.9780898
[43] 0.9777175 0.9753188 0.9768748 0.9772850 0.9817677 0.9785937 0.9813918
[50] 0.9779363

[1] 0.02309125 0.02519580 0.02481980 0.02577179 0.02526503 0.02767727
 [7] 0.02246305 0.02365940 0.02551210 0.02076892 0.02517405 0.02422078
[13] 0.02167973 0.02774275 0.02331052 0.02978442 0.02818439 0.02087170
[19] 0.03093554 0.02345981 0.02854512 0.02403941 0.02694905 0.02611122
[25] 0.03074649 0.02203611 0.02099528 0.02070863 0.02124002 0.02027494
[31] 0.02775911 0.02740260 0.02148557 0.03185550 0.02581281 0.02592270
[37] 0.02921161 0.03155310 0.02553714 0.02653463 0.02259740 0.02746753
[43] 0.02774465 0.03012994 0.02846334 0.02836403 0.02384236 0.02723125
[49] 0.02391304 0.02765189

[1] 0.12265673 0.14225002 0.13477492 0.13450292 0.14187774 0.16409573
 [7] 0.11198686 0.12778478 0.13811297 0.13032666 0.15332776 0.12772727
[13] 0.10744833 0.14399808 0.11804962 0.15889029 0.16018233 0.10270104
[19] 0.17664309 0.12403101 0.16261510 0.12486043 0.14837324 0.14211680
[25] 0.18159645 0.11540746 0.10271743 0.09907121 0.11375077 0.09936602
[31] 0.16514316 0.14889610 0.11467158 0.17359606 0.13799672 0.15553618
[37] 0.16519004 0.17753223 0.15076734 0.13984638 0.11720779 0.15227273
[43] 0.15303496 0.16897872 0.16495526 0.16279070 0.12722496 0.15084117
[49] 0.12344301 0.14660099

In [29]:
# Finally, print the efficiency score and connectivity scores for the Autism Interactome
efficiency(get.adjacency(HS.stree,sparse=F))
c.scores(HS.stree)

[1] 0.9723703

[1] 0.03438228 0.17686480

# Identifying New Candidate Genes

In [30]:
# compute the betweeness centrality scores for each node
betweeness_centrality_scores = igraph::betweenness(HS.stree)
# now identify only those NOT already known to be significant
significant_centrality = c()
count = 0
for (i in 1:length(betweeness_centrality_scores)){
if (!(as.numeric(names(betweeness_centrality_scores[i])) %in% signif.ids)) {
significant_centrality = c(significant_centrality,
betweeness_centrality_scores[i])
}
}

In [31]:
# Sorting
significant_centrality = sort(significant_centrality, decreasing=TRUE)
length(significant_centrality)

[1] 49